In [6]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

# anxiety_label_type = "Anxiety"
anxiety_label_type = None
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "DT": DecisionTreeClassifier(criterion="gini", max_features="sqrt"),
    "LogReg": LogisticRegression(max_iter=1000),
    "RF": RandomForestClassifier(n_estimators=100, max_features="sqrt"),  #  min_samples_split=3
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="error")
}

threshold = "fixed"
asc_threshold = "fixed"
test_size = 1.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))


In [8]:
# TRAIN ON APD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_asc)
importlib.reload(dt)


print(f"APD PHASES {model_phases_apd} " + "-"*50)
x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
# drop subjects with noisy data
x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)]
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)]
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0)
y_b = y_b.drop(labels=inds, axis=0)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

APD PHASES ['Baseline_Rest', 'BugBox_Relax', 'BugBox_Anticipate', 'BugBox_Exposure', 'BugBox_Break', 'Speech_Relax', 'Speech_Anticipate', 'Speech_Exposure', 'Speech_Break'] --------------------------------------------------
y_train:
0    1142
1     890
Name: label, dtype: int64
y_test:
1    3001
0    1540
Name: label, dtype: int64
Model KNN, Predictions: [0 1], [1149 3392]
Feature importance not available for KNN
Model DT, Predictions: [0 1], [1758 2783]
Model LogReg, Predictions: [0 1], [ 174 4367]
Model RF, Predictions: [0 1], [ 669 3872]
Model XGB, Predictions: [0 1], [1651 2890]

zip argument #2 must support iteration


[('mean_SCL', 0.19368459300739949), ('bpm', 0.15945976508410087), ('lf_rr', 0.12574735513410915), ('hf_rr', 0.11948270676239811), ('sdnn', 0.10742498853063151), ('rmssd', 0.10471554154296471), ('SCR_rate', 0.10304592691256645), ('lf_hf_ratio', 0.08643912302582955)]


[('mean_SCL', 1.859662622363344), ('hf_rr', 1.2283765448850408), ('lf_rr', 0.7836914360112125), ('sd

In [9]:
# TRAIN ON ASCERTAIN AND TEST ON APD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_asc)
importlib.reload(dt)


x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
# x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

y_train:
1    3001
0    1540
Name: label, dtype: int64
y_test:
0    1142
1     890
Name: label, dtype: int64
Model KNN, Predictions: [0 1], [1065  967]
Feature importance not available for KNN
Model DT, Predictions: [0 1], [1318  714]
Model LogReg, Predictions: [0 1], [ 354 1678]
Model RF, Predictions: [0 1], [1056  976]
Model XGB, Predictions: [0 1], [1351  681]

zip argument #2 must support iteration


[('mean_SCL', 0.17182180316320844), ('bpm', 0.15756029833448815), ('lf_hf_ratio', 0.14012915587844132), ('sdnn', 0.13598356162723307), ('rmssd', 0.12888133445172342), ('hf_rr', 0.11334077762021647), ('lf_rr', 0.10919581964661249), ('SCR_rate', 0.043087249278076584)]


[('SCR_rate', 0.32136064076511145), ('mean_SCL', 0.16795083666647767), ('hf_rr', -0.08106088974854966), ('lf_hf_ratio', -0.11289343501967576), ('bpm', -0.299761021414726), ('lf_rr', -0.33511538607025276), ('rmssd', -0.965536298357176), ('sdnn', -1.2139235835099742)]


[('mean_SCL', 0.16453166423541418), ('bpm', 0.15156646

In [10]:
# TRAIN ON ASCERTAIN AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_asc)
importlib.reload(dt)


# for j, phases_apd in enumerate(model_phases_apd):
for j, phases_apd in enumerate([dr_a.Phases.BUG_EXPOSURE, dr_a.Phases.SPEECH_EXPOSURE]):
    print(f"APD PHASES {phases_apd} " + "-"*50)
    phases_apd = [phases_apd]
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
    x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
    y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

    if anxiety_label_type is not None:
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        "KNN": [],
        "DT": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        "KNN": [],
        "DT": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        print(f"Model evaluation metrics for {model_name}:")
        for i in range(len(reports[model_name])):
            report = reports[model_name][i]
            acc = acc_results[model_name][i]
            p = report["precision"]
            r = report["recall"]
            f1 = report["f1"]
            auc = report["auc"]
            print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
        print("")
    print("\n")

APD PHASES BugBox_Exposure --------------------------------------------------
Ratio of negative to positive labels (0.25) is under 0.333, oversampling negative class.
y_train:
1    3001
0    1540
Name: label, dtype: int64
y_test:
1    160
0    160
Name: label, dtype: int64
Model KNN, Predictions: [0 1], [191 129]
Feature importance not available for KNN
Model DT, Predictions: [0 1], [136 184]
Model LogReg, Predictions: [0 1], [135 185]
Model RF, Predictions: [0 1], [208 112]
Model XGB, Predictions: [0 1], [212 108]

zip argument #2 must support iteration


[('rmssd', 0.16116094876864545), ('mean_SCL', 0.15749302563103898), ('sdnn', 0.1502544098202367), ('bpm', 0.13398728556496983), ('lf_hf_ratio', 0.1305531809034307), ('lf_rr', 0.11838775390916378), ('hf_rr', 0.11121902667030734), ('SCR_rate', 0.03694436873220728)]


[('SCR_rate', 0.32136064076511145), ('mean_SCL', 0.16795083666647767), ('hf_rr', -0.08106088974854966), ('lf_hf_ratio', -0.11289343501967576), ('bpm', -0.299761021414726),